<center><img src="logo.png" alt="Header" style="width: 800px;"/></center>

In [1]:
# 导入库  import library
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display
import logging
import time
import threading
import inspect
import ctypes
import numpy as np
import pyzbar.pyzbar as pyzbar

import random
from enum import Enum, auto

In [2]:
# Dogzilla drive library
from DOGZILLALib import DOGZILLA
g_dog = DOGZILLA()

# 二维码识别 QRCode recognition

In [3]:
# 中文开关，默认为英文 Chinese switch. The default value is English
g_ENABLE_CHINESE = False

Name_widgets = {
    'Close_Camera': ("Close_Camera", "关闭摄像头")
}

In [4]:
# 图像数据转化  Image data transformation
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [5]:
# 关闭线程  stop thread
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [6]:
# 创建摄像头显示组件  Create the camera display component
image_widget = widgets.Image(format='jpeg', width=640, height=480)  

# 打开摄像头，数字0需根据/dev/videoX修改为X
# Turn on the camera, you need to change the number 0 to X based on /dev/videoX
image = cv2.VideoCapture(0)
image.set(3, 640)
image.set(4, 480)
image.set(5, 30)
image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))

True

In [7]:
# 关闭摄像头 Close_Camera
button_Close_Camera = widgets.Button(  
    value=False,  
    description=Name_widgets['Close_Camera'][g_ENABLE_CHINESE],      
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )


# 按键按下事件处理   Key press event processing
def on_button_close_camera(b):
    if b.description == Name_widgets['Close_Camera'][g_ENABLE_CHINESE]:
        # 停止线程，释放摄像头  Stop the thread and release the camera
        b.icon = 'uncheck'
        stop_thread(thread1)
        image.release()
    
# 关联按键事件回调 Button event callbacks
button_Close_Camera.on_click(on_button_close_camera)

In [8]:
commands_array = []

#with open('fileName.txt', 'r') as file:
    #commands = file.readlines()
    #commands_array.append(commands)

In [9]:
# global variables for movements 
bool_walk = True # walk forward or not
bool_spin = False # decide to spin (turnleft)
bool_scan = False # determines if qr code is read or not ; scanning QR code
bool_back = False 
count = 0 # count for spins
bool_turned_walk = False

# storing variables
dogzilla_behavior = 1
adjacent_nodes = set() # stores adjacent nodes 
current_node_location = "" 
previous_node = ""
last_seen_node = ""
distance = 0 # distance until qr is seen - note: use LIDAR
bool_determine_path = True # options in the Flow chart
time_start = time.time()

In [10]:
# 解析图像中的二维码信息  Analyze the qrcode information in the image
import time ##So we can use .sleep()


class MovementState(Enum):
    IDLE = auto()
    WALKING_BACK = auto()
    WALKING_FORWARD = auto()
    SCANNING = auto()
    SPINNING = auto()
    DETERMINING_PATH = auto()
    #New for 4-14-25
    NEARING_WALL = auto()
    ALIGNING_QR = auto()

class MovementSystem:
    def __init__(self, g_dog, commands_array):
        self.g_dog = g_dog
        self.state = MovementState.WALKING_FORWARD #MovementState.WALKING_FORWARD <-- Have this once done testing
        self.start_time = time.time()
        self.current_node_location = None
        self.previous_node = None
        self.bool_determine_path = True #False
        self.last_seen_node = None
        self.commands_array = commands_array
        self.target_command = commands_array[0]
        self.adjacent_nodes = set()
        #Have a finsihed Identifier?
        self.completed_maze = False
        self.alignment = []

    def get_state(self):
        _curr_state = ""
        if self.state == MovementState.IDLE:
            return "IDLE"
        elif self.state == MovementState.WALKING_BACK:
            return "WALKING_BACK"
        elif self.state == MovementState.WALKING_FORWARD:
            return "WALKING_FORWARD"
        elif self.state == MovementState.SCANNING:
            return "SCANNING"
        elif self.state == MovementState.SPINNING:
            return "SPINNING"
        elif self.state == MovementState.DETERMINING_PATH:
            return "DETERMINING_PATH"

    def get_current_location(self):
        return self.current_node_location
    def get_previous_node(self):
        return self.previous_node
    def get_bool_determine_path(self):
        return self.bool_determine_path
    def get_target_command(self):
        return self.target_command
    def get_commands_array(self):
        return self.commands_array
    
    def get_alignment(self):
        return self.alignment
    
    def update(self, barcodes, barcodeData, center_x=None, frame_width=None):
        end_time = time.time()
        elapsed_time = end_time - self.start_time

        #If there are no commands left, stop the robot
        #Maybe make it so the state is either in SCANNING OR SPINNING?
        if not self.commands_array and self.state == MovementState.WALKING_FORWARD and self.completed_maze: #test
            self.g_dog.stop()
            print("Maze Completed! Stopping Robot.")
            return #Just so that the robot doesnt do anything else

            
        # if self.state == MovementState.IDLE: #The orignal before edits on 4-14-25
        #     if len(barcodes) == 0 and elapsed_time > 2: #could do 3 too --> was 2
        #         self.g_dog.stop()
        #         self.g_dog.back(5)
        #         self.state = MovementState.WALKING_BACK
        #         print("I moved backwards")
        #         self.start_time = time.time()
        
        if self.state == MovementState.IDLE:
            if len(barcodes) == 0 and elapsed_time > 2: #could do 3 too --> was 2
                ###self.g_dog.stop()
                #self.g_dog.back(5)
                self.state = MovementState.NEARING_WALL
                print("I switched to NEARING_WALL")
                self.start_time = time.time()
            elif len(barcodes) != 0:
                self.g_dog.forward(5)
                self.start_time = time.time()
                ##--> self.start_time = time.time() #If this new thing doesnt work

        elif self.state == MovementState.NEARING_WALL: #New for 4-14-25
            if len(barcodes) != 0: #Meaning it has seen a QR Code
                self.state = MovementState.IDLE
                self.g_dog.forward(5) # Might reduce to 2 or 3 --> twas 5
                self.start_time = time.time()
            #Note for Future after 4-14-25 -->Maybe just when we hit the wall for 5 seconds that we go immediately into scanning?
            elif len(barcodes) == 0 and elapsed_time > 3.7:#5: #The maxium tolerance of looking at wall and deciding what to do next.
                self.g_dog.stop()
                ###self.g_dog.back(5) #Since we determined that we are at the corner
                self.state = MovementState.SCANNING #So it goes to wall and just turns without having to move back first
                self.start_time = time.time()
                
        elif self.state == MovementState.WALKING_BACK: #04/14/Thoughts: Do we want this or just transfer NEARING_WALL logic to here.
            if len(barcodes) == 0: #if barcodeData != None: #Do i need to check for Barcode len != 0 for all of these below? (Hoping lazy approach works)
                self.state = MovementState.WALKING_FORWARD
                self.g_dog.forward(5)
                print("I walk forward after seeing a QR code")

        # elif self.state == MovementState.WALKING_FORWARD: #Changing on 4/8/25 --> THis is Original before 4/8
        #     self.state = MovementState.SCANNING
        #     self.g_dog.stop()
        #     self.current_node_location = barcodeData
        #     self.previous_node = self.current_node_location
        #     print("Scanning QR code")

        #Note for Future after 4-14-25 -->Maybe just when we hit the wall just skip this state and go immediately into spinning?
        elif self.state == MovementState.WALKING_FORWARD: #Changing on 4/8/25
            if len(barcodes) != 0: #meaning that we now saw a QR Code
                self.g_dog.stop()
                self.state = MovementState.SCANNING
                self.current_node_location = barcodeData
                self.previous_node = self.current_node_location
            #Maybe add else statement to have it walk forwards until it sees QR code and then Trigger IDLE State
            print("Scanning QR code")

        elif self.state == MovementState.SCANNING: #Should Turn Logic Be here?
            #Adding a stop statement here for Safety --> 4-10-25
            self.g_dog.stop()
            # The first barcode scanned determines our turn direction
            if self.target_command == "left":
                print("Target command is LEFT. Turning left.")
                self.g_dog.turnleft(1) #WAS 10 --> Was 3
            elif self.target_command == "right":
                print("Target command is RIGHT. Turning right.")
                self.g_dog.turnright(1) #WAS 10 --> Was 3

            #Should I include a self.target_command == None: scenario? 4-10-25

            self.state = MovementState.SPINNING
            self.adjacent_nodes.clear()  # Reset adjacent nodes
            print("Entering SPINNING state to detect new node.")

        elif self.state == MovementState.SPINNING:
            # Collect all QR codes seen during spinning
            if barcodeData and barcodeData != self.current_node_location:
                self.adjacent_nodes.add(barcodeData)

            # Stop when detecting the first new QR code, Since we now dont care about finding or Initializing Tree
            # We now need to react off of first visuals and commands
            ### Additionally, Check if the QR code is centered (within a margin of error)!!
            if center_x is not None and frame_width is not None:
                center_margin = 40  # pixels of acceptable offset from center --> was 40 --> Tried 15
                frame_center = frame_width // 2
                
                if len(self.adjacent_nodes) > 0 and abs(center_x - frame_center) <= center_margin:# and barcodeData != self.current_node_location: #Adding on 4/8/25
                    self.g_dog.stop()
                    self.current_node_location = list(self.adjacent_nodes)[0] # Take the first detected node #Changing on 4/8/25
                    print(f"Detected new node: {self.current_node_location}, moving forward.")

                    #Need to check if its not in center of margin:
                    
                    # if self.target_command == "left":
                    #     self.g_dog.turnright(1)
                    # elif self.target_command == "right":
                    #     self.g_dog.turnleft(1)                
                    
                    #was --> if True: ###if abs(center_x - frame_center) <= (center_margin-15):
                    if abs(center_x - frame_center) <= (center_margin-15):
                        self.g_dog.stop()
                        #self.state = MovementState.WALKING_FORWARD #TEMP --> Should this be be set after checking if commands list is empty?(To Thonk abt)
                        ####self.state = MovementState.DETERMINING_PATH
                        self.state = MovementState.ALIGNING_QR
                        self.start_time = time.time()
                        #Checking that even if it is empty, it still needs to execute the last command
                        #Thus we need to keep the last command untouched, and pop() on an empty list throws error & problems
                        if not self.commands_array:
                            self.completed_maze = True
                        else:
                            self.commands_array.pop(0)  # Remove executed command
                            self.target_command = self.commands_array[0] if self.commands_array else None  # Update target command
    
                        ###self.g_dog.forward(10)
                        self.adjacent_nodes.clear()
                    else:
                        if self.target_command == "left":
                            self.g_dog.turnright(1)
                            #self.g_dog.stop()
                        elif self.target_command == "right":
                            self.g_dog.turnleft(1)
                            #self.g_dog.stop()

        elif self.state == MovementState.DETERMINING_PATH:
            self.bool_determine_path = True
            self.state = MovementState.IDLE

        elif self.state == MovementState.ALIGNING_QR: #This is for when we at node and want to re center after seeing where we want to go!
            if len(barcodes) > 0:
                barcode = barcodes[0]  # Just use the first one
                (x, y, w, h) = barcode.rect
                qr_center_x = x + w // 2
                self.alignment.append(len(barcodes))
        
                frame_center_x = 640 // 2  # Assuming 640x480 resolution
                margin = 15  # Pixel range for acceptable center alignment --> was 30 --> Had it at 15 but we test
        
                if qr_center_x < frame_center_x - margin:
                    print("QR too far left. Crab-walking right.")
                    self.alignment.append("QR too far left. Crab-walking right.")
                    #self.g_dog.right(1)
                    self.g_dog.turnright(1)
                    ##self.g_dog.stop()##Remove these stops
                elif qr_center_x > frame_center_x + margin:
                    print("QR too far right. Crab-walking left.")
                    self.alignment.append("QR too far right. Crab-walking left.")
                    #self.g_dog.left(1)
                    self.g_dog.turnleft(1)
                    ##self.g_dog.stop() ##Remove these stops
                else:
                    print("QR code is centered. Proceeding to SCANNING.")
                    self.alignment.append("QR code is centered. Proceeding to SCANNING.")
                    self.g_dog.stop()
                    self.g_dog.forward(5) #was 10
                    self.state = MovementState.IDLE
            else:
                print("Lost sight of QR during ALIGNING_QR. Returning to IDLE.")
                self.alignment.append("QR code is centered. Proceeding to SCANNING.")
                self.state = MovementState.IDLE
                #Mayhaps
                self.g_dog.stop()

        #Cuz Logan is Stupid
        if len(barcodes) != 0:
            self.last_seen_node = self.previous_node
            # updates previous_node
            self.previous_node = barcodeData

In [11]:
#We will be acquiring the Path to Take from the Virtual Team
#WE need to confirm if they are giving us a text file of the names on a new line or if they give us the array straight up
#Example:
# Node 1
# Node 7
# Node 4
# Then while loop it into a array/list

#For now we hardcode path


starting_yaw = g_dog.read_imu_raw()[8]
print("Initial yaw: ", starting_yaw)

Initial yaw:  -309.2986881510417


In [12]:
# Initial walk cycle
if bool_walk:
    g_dog.stop()
    ####g_dog.forward(10) # 70 is default
    print("I walked")
    # start timer once it walks to handle deadend
    start_time = time.time()

commands_array = ["right", "left"] #Just to test

###commands_array = ["right", "right", "left"] #Just to test
##commands_array = ["left", "right", "left", "left", "right", "left"] #For testing Maze Set up by Rose and Nav on 4-14-25

####LOGAN SWITCHING EVERYTHING TO MOVEMENT SYSTEMS
# Initialize movement system
movement_system = MovementSystem(g_dog, commands_array)

def decodeDisplay(image, display):
    logging.debug('Barcode read')
    
    barcodes = pyzbar.decode(image)
    
    center_x = None
    if len(barcodes) == 0:
        barcodeData = None
        movement_system.update(barcodes, barcodeData, center_x, display.shape[1]) ##display.shape[1]
        ##movement_system.update(barcodes, barcodeData, center_x=center_x, frame_width=display.shape[1]) <-- before Testing on 4-10-25
    
    for barcode in barcodes:
        # 提取二维码的边界框的位置, 画出图像中条形码的边界框
        # Extract the position of the bounding box of the qrcode, 
        # and draw the bounding box of the barcode in the image
        (x, y, w, h) = barcode.rect
        cv2.rectangle(display, (x, y), (x + w, y + h), (225, 225, 225), 2)
        #04-08-25 Acquiring the Center
        center_x = x + w // 2
        
        # 提取二维码数据为字节对象，转换成字符串
        # The qrcode data is extracted as byte objects and converted into strings
        barcodeData = barcode.data.decode("utf-8")
        barcodeType = barcode.type

        # 绘出图像上条形码的数据和条形码类型  
        # Plot the barcode data and barcode type on the image
        text = "{} ({})".format(barcodeData, barcodeType)
        cv2.putText(display, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (225, 0, 0), 2)

        #04-08-25 --> Moving Movement System being called in the event that its 
        #affecting the rendiering of the Text and border of QR Code
        movement_system.update(barcodes, barcodeData, center_x, display.shape[1]) ##display.shape[1]
        ##movement_system.update(barcodes, barcodeData, center_x=center_x, frame_width=display.shape[1]) <-- before Testing on 4-10-25
    return display
####END OF LOGAN SWITCHING EVERYTHING TO MOVEMENT SYSTEMS




# 检测二维码  detect qrcode
def Detect_Qrcode_Task():
    ret, frame = image.read()
    image_widget.value = bgr8_to_jpeg(frame)
    t_start = time.time()
    fps = 0
    while True:
        ret, frame = image.read()
        # 转为灰度图像  Convert to grayscale image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = decodeDisplay(gray, frame)
        fps = fps + 1
        mfps = fps / (time.time() - t_start)
        cv2.putText(frame, "FPS " + str(int(mfps)), (40,40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 3)
        image_widget.value = bgr8_to_jpeg(frame)

I walked


In [13]:
# 启动摄像头显示任务  Start the camera display task
thread1 = threading.Thread(target=Detect_Qrcode_Task)
thread1.setDaemon(True)
thread1.start()

box_display = widgets.HBox([image_widget, button_Close_Camera])
display(box_display)

In [14]:
#g_dog.stop() #-> Just in case the robot won't stop#

In [15]:
print(current_node_location)
print(list(adjacent_nodes))
print(bool_walk)
print(bool_scan)
print(bool_determine_path)
print(last_seen_node)
print(previous_node)
print(bool_back)


[]
True
False
True


False


In [16]:
_current_state = movement_system.get_state()
print(_current_state)

WALKING_FORWARD


In [17]:
_curr_location = movement_system.get_current_location()
_prev_node = movement_system.get_previous_node()
_bool_dermine_path = movement_system.get_bool_determine_path()
_target_command = movement_system.get_target_command()
_commands_array = movement_system.get_commands_array()

print("Current Node: ", _curr_location)
print("Previous Noe: ", _prev_node)
print("Bool Determine Path: ", _bool_dermine_path)
print("Target Command: ",_target_command)
print("Path to Take: ", _commands_array)

Current Node:  None
Previous Noe:  None
Bool Determine Path:  True
Target Command:  right
Path to Take:  ['right', 'left']


In [18]:
#Printing out IMU DATA
#while(True):
    #temp_imu = g_dog.read_imu_raw()[8]
    #print(temp_imu)

In [19]:
# align_list = movement_system.get_alignment()
# for req in align_list:
#     print(req)